# RSI STRATEGY

The RSI is a widely used technical indicator and an oscillator that indicates a market is overbought when the RSI value is over 70 and indicates oversold conditions when RSI readings are under 30. Some traders and analysts prefer to use the more extreme readings of 80 and 20. A weakness of the RSI is that sudden, sharp price movements can cause it to spike repeatedly up or down, and, thus, it is prone to giving false signals.

In [1]:
%matplotlib inline  

import pandas as pd
import talib as tl
import numpy as np

In [2]:
def load(file):
    df = pd.read_csv(file, skiprows=1)
    return df.reindex(index=df.index[::-1]).reset_index()

In [3]:
datasets = {
    "BTCUSD": load('data/Bitstamp_BTCUSD_1h.csv'),
    "ETHUSD": load('data/Bitstamp_ETHUSD_1h.csv'),
    "LTCUSD": load('data/Bitstamp_LTCUSD_1h.csv'),
    "XRPUSD": load('data/Bitstamp_XRPUSD_1h.csv'),
    "ETHBTC": load('data/Binance_ETHBTC_1h.csv'),
    "LTCBTC": load('data/Binance_LTCBTC_1h.csv'),
    "XMRBTC": load('data/Binance_XMRBTC_1h.csv')
}

In [4]:
def benchRSI(df, lowRSI = 30.0, highRSI = 70.):
    data = df.copy()
    data['RSI'] = tl.RSI(data.Close, timeperiod = 14)
    data = data[(data.RSI < 30.) | (data.RSI > 70.)]

    def advice(rsi):
        if rsi<30:
            #buy
            return 'long'
        if rsi>70:
            #sell
            return 'short'
        return np.nan

    data['advice'] = data['RSI'].apply(advice)
    data['prevAdvice'] = data['advice'].shift()
    data = data[data.prevAdvice != data.advice][['Date', 'Close', 'RSI', 'advice']]

    data['enterprice'] = data.Close.shift()

    if data.advice.values[0] == 'short':
        data = data[1:]

    data = data.dropna()
    deals = data[data.advice == 'short'].copy()
    deals['earn'] = (deals.Close - deals.enterprice) / deals.enterprice
    data.drop(['enterprice'], axis=1)
    
    return deals.earn, data

In [40]:
def computePositions(data):
    rows = []
    positionTake = False
    for i, e in data.iterrows():

        if e.advice == 'short' and positionTake:
            # close position
            rows.append({
                "enter": position.Date,
                "exit": e.Date,
                "buyprice": position.Close,
                "sellprice": e.Close,
                "profit": (e.Close - position.Close) / position.Close,
            })
        else:
            # open position
            positionTake = True
            position = e
    return pd.DataFrame(rows)

In [42]:
results = {}
for e in datasets:
    roi, log = benchRSI(datasets[e])
    results[e] = { "roi": roi, "roiAvg": roi.mean(), "log": log, "positions": computePositions(log) }
    

In [43]:
for e in results:
    print(e)
    print('ntrades', len(results[e]['roi']), ', mean roi',results[e]['roiAvg'])

BTCUSD
ntrades 51 , mean roi 0.0026732290726156184
ETHUSD
ntrades 41 , mean roi -0.035216098055810066
LTCUSD
ntrades 52 , mean roi 0.009105792318809352
XRPUSD
ntrades 50 , mean roi -0.012605270973493005
ETHBTC
ntrades 57 , mean roi -0.02249376560882072
LTCBTC
ntrades 53 , mean roi -0.006085037037699648
XMRBTC
ntrades 39 , mean roi 0.025003177837482283


In [45]:
results["BTCUSD"]["positions"]

,buyprice,enter,exit,profit,sellprice
0,2524.98,2017-07-07 01-PM,2017-07-17 10-PM,-0.125161,2208.95
1,2600.06,2017-07-25 10-AM,2017-07-27 10-PM,0.031349,2681.57
2,4090.00,2017-08-18 06-PM,2017-08-24 10-PM,0.054807,4314.16
3,4240.00,2017-08-28 04-AM,2017-08-29 01-PM,0.086743,4607.79
4,4066.51,2017-09-04 05-PM,2017-09-12 07-AM,0.071403,4356.87
5,4026.02,2017-09-13 02-AM,2017-09-18 07-AM,-0.020124,3945.00
6,3690.15,2017-09-21 04-PM,2017-11-12 05-AM,0.592076,5875.00
7,7989.00,2017-11-23 11-PM,2017-11-25 11-AM,0.054321,8422.97
8,12813.34,2017-12-09 05-PM,2017-12-11 07-AM,0.279916,16400.00
9,17759.44,2017-12-19 11-AM,2017-12-26 06-AM,-0.123805,15560.74
